# Stream and Block Ciphers

This notebook solves the problems presented in "Implementation Cryptography with Python" by S.Bray"Implementation Cryptography with Python" by S.Bray for a simple stream cipher.

### Stream Ciphers

#### A simple (unsecure) stream cipher. This is composed of a random number generator to create a shared private key, encryption (XOR), and decryption (XOR).

In [276]:
import binascii

In [277]:
def crand(seed):
    ''' 8 bit random number generator.
    INPUT: 
        seed - the starting point the generator.
    OUTPUT: 
        an integer random number.
    '''
    
    # Copy of a pseudorandom number generator.
    r=[]
    r.append(seed)
    for i in range(30):
        r.append((16807*r[-1]) % 2147483647)
        if r[-1] < 0:
            r[-1] += 2147483647    
    for i in range(31, 34):
        r.append(r[len(r)-31])
    for i in range(34, 344):
        r.append((r[len(r)-31] + r[len(r)-3]) % 2**32)

    while True:
        next = r[len(r)-31]+r[len(r)-3] % 2**32
        r.append(next)
        yield (next >> 1 if next < 2**32 else (next % 2**32) >> 1)

In [285]:
def encrypt(msg, key):
    ''' Encrypt a message with a private key.
        The message and key will be converted to hex, the hex then to int base 16, finally an xor applied. In python xor on ints is done at bit level.
        INPUTS:
            msg - plaintext message to encrypt.
            key - random integer key with same number of bytes as message.
        OUTPUT:
            cipherHex - the cipher in hex format.
    '''

    # Convert plain text in (byte) form to hex.
    hexPlain = binascii.hexlify(plainText)
    
    # Convert key to hex. In the code below the key
    # is a list. The map function passes the elements of the list
    # into the lambda function. The join assembles it as one string.
    # Remember the map is an iterable.
    keyHex = "".join(map(lambda x: format(x, 'x')[-6:], key)) # Only use 6 bits ASCII.
    
    # Apply XOR operation (OTP like). To do this need to convert to int.
    cipherInt = int(hexPlain, 16) ^ int(keyHex, 16)
    
    # Return cipher text as hex.
    cipherHex = format(cipherInt, 'x')
    
    return cipherHex

In [279]:
def decrypt(cipherHex, key):
    ''' Decrypt a message with a private key.
        INPUTS:
            cipherHex - hex encoded cipher.
            key - random integer key with same number of bytes as message.
        OUTPUT:
            cipherHex - the cipher in hex format.
    '''
    
    # Convert the hex back to an int for xor operation.
    cipherInt = int(cipherHex, 16)

    # Convert key to hex. In the code below the key
    # is a list. The map function passes the elements of the list
    # into the lambda function. The join assembles it as one string.
    # Remember the map is an iterable.
    keyHex = "".join(map(lambda x: format(x, 'x')[-6:], key)) # Only use 6 bits ASCII.

    # Decrypt by applying XOR operation again.
    msgHex = format(cipherInt ^ int(keyHex, 16), 'x')

    # Convert back to plain text.
    msgPlain = binascii.unhexlify(msgHex)

    return msgPlain

##### Now use the above functions to demonstrate on a simple message.

In [280]:
# Define the message to send.
plainText = b"Hello World Ending"
# Create the key which is really just the seed.
seed = 2018
mygen = crand(seed)
key = [next(mygen) for i in range(4)] # Create a key which is at least as long as message in terms of bits.
print("The key is {}".format(key))

# Encrypt the message.
cipherHex = encrypt(plainText, key)

# Decrypt the message.
msgDecrypt = decrypt(cipherHex, key)
print(msgDecrypt)

The key is [1471611625, 1204518815, 463882823, 963005816]
b'Hello World Ending'


Take the seed (key) and cipherHex in the example where the decryption is not given and try to get the right answer!

In [281]:
# Create new key.
seed = 54321
mygen = crand(seed)
key = [next(mygen) for i in range(6)] # NB MUST BE SAME LENGTH AS CIPHER HEX FOR OTP!!!!

print("The key is {}".format(key))

# Defined cipher text - with unknown message.
cipherHex = 'e5d8443c6ac32d3ee5c7398ecf7f9e03f619'

# Decrypt the message.
msgDecrypt = decrypt(cipherHex, key)
print(msgDecrypt)

The key is [60209456, 357898661, 257185675, 1235229180, 765860086, 1920452902]
b'satisfying right??'


### Can this be improved (at a trivial level)?

We could use a nonce/"Initialisation Vector" to modify the key. We would send the nonce in the clear every time with the message. However this nonce would be used to generate a new seed for the secret key. This would add some randomness in case we sent the same message several times. The following code from S. Bray shows how.

In [282]:
import os
import hashlib

# Create a random nonce. This will be appended to the cipher text - it will all look random but both sides know where the nonce is.
nonce = os.urandom(6) # This function uses some OS randomness.
# Create a hex string from the random bytes.
hexnonce = binascii.hexlify(nonce).decode('ascii')

# There is still a shared private secret.
oursecret = 9123435

# However, we now hash (one way function) the nonce and our secret. The hashed value is then used to create the key.
concatenated_hex = hexnonce + format(oursecret, 'x')
even_length = concatenated_hex.rjust(len(concatenated_hex)+len(concatenated_hex)%2,'0')
hexhash = hashlib.sha256(binascii.unhexlify(even_length)).hexdigest()

# Now create a new random seed that can be used for the encryption and decryption.
newseed = (int (hexhash, 16)) % 2**32

# Not shown but the hexnonce should be appended to the cipher text so the receiver can decrypt. See the problem below.

To prove understanding take the secret key and cipher text that S. Bray gives and try to decrypt!

In [284]:
# We are told first 6 bytes are nonce bytes.
cipherTxt = '3e08816f1377f89f1c596fc197dd52946c92577bfd7c25c3'

# We are told the secret key.
oursecret = 61983

# Extract the nonce and message.
nonceHex = binascii.unhexlify(cipherTxt)[0:6]
msgHex = binascii.unhexlify(cipherTxt)[6:]

# Now find the new seed.
hexnonce = binascii.hexlify(nonceHex).decode('ascii')
# However, we now hash (one way function) the nonce and our secret. The hashed value is then used to create the key.
concatenated_hex = hexnonce + format(oursecret, 'x')
even_length = concatenated_hex.rjust(len(concatenated_hex)+len(concatenated_hex)%2,'0')
hexhash = hashlib.sha256(binascii.unhexlify(even_length)).hexdigest()

# Now create a new random seed that can be used for the encryption and decryption.
newseed = (int (hexhash, 16)) % 2**32

# Find the key that was used.
mygen = crand(newseed)
key = [next(mygen) for i in range(6)] # NB MUST BE SAME LENGTH AS CIPHER HEX FOR OTP!!!!

# Decrypt the message.
msgDecrypt = decrypt(binascii.hexlify(msgHex), key)
print(msgDecrypt)

b'this is a message.'
